Beat-n game with Nash-env derived from 10 pt limit and augmented with higher-cost sub-Nash Spuds.

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [3]:
const ability_none = 999
const ability_imitative = 10
const ability_reciprocating = 20
const ability_romantic = 30
const ability_bibliophile = 40
const ability_melodious = 50
const ability_iconoclast = 90
const ability_intoxicating = 100
const ability_otaku = 110
const ability_sincere = 120
const ability_cowardly = 130


ability_name = Dict(
    ability_imitative => "Imitative*",
    ability_reciprocating => "Reciprocating*",
    ability_romantic => "Romantic*",
    ability_bibliophile => "Bibliophile*",
    ability_melodious => "Melodious*",
    ability_iconoclast => "Iconoclast*",
    ability_intoxicating => "Intoxicating*",
    ability_otaku => "Otaku*",
    ability_sincere => "Sincere*",
    ability_cowardly => "Cowardly*",
)

function fx_imitative(a::Spud, b::Spud)
    if b.h > a.h
        a = Spud(a.name, b.h, a.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_reciprocating(a::Spud, b::Spud)
    if b.f > a.f
        a = Spud(a.name, a.h, b.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_romantic(a::Spud, b::Spud)
    if b.p > a.l
        a = Spud(a.name, a.h, a.f, b.p, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_bibliophile(a::Spud, b::Spud)
    if b.r > a.l
        a = Spud(a.name, a.h, a.f, b.r, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_melodious(a::Spud, b::Spud)
    if b.p > b.r && b.p > b.s
        a = Spud(a.name, a.h, a.f, a.r, a.p + 3, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_iconoclast(a::Spud, b::Spud)
    v = sum([b.h,b.f,b.l,b.p,b.r,b.s] .== MXV)
    if v >= 3
        aa = [a.h, a.f, a.l, a.p, a.r, a.s]
        aa2 = map(x -> div(x, 4), aa)
        r = aa .+ aa2
        a = Spud(a.name, r[1], r[2], r[3], r[4], r[5], r[6], a.a1)
    end
    return [a, b]
end

function fx_intoxicating(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.r, b.f, b.a1)
    return [a, b]
end

function fx_otaku(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.h, b.s, b.a1)
    return [a, b]
end


function fx_sincere(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f + 2, b.f + 2, b.p, b.r, b.s, b.a1)
    return [a, b]
end

function fx_cowardly(a::Spud, b::Spud)
    if (a.p < b.p) || (a.r < b.r) || (a.s < b.s)
        a = Spud(a.name, a.h + 2, a.f - 2, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

ability_fx = Dict(
    ability_imitative => fx_imitative,
    ability_reciprocating => fx_reciprocating,
    ability_romantic => fx_romantic,
    ability_bibliophile => fx_bibliophile,
    ability_melodious => fx_melodious,
    ability_iconoclast => fx_iconoclast,
    ability_intoxicating => fx_intoxicating,
    ability_otaku => fx_otaku,
    ability_sincere => fx_sincere,
    ability_cowardly => fx_cowardly,
)

Dict{Int64, Function} with 10 entries:
  50  => fx_melodious
  20  => fx_reciprocating
  110 => fx_otaku
  10  => fx_imitative
  90  => fx_iconoclast
  30  => fx_romantic
  120 => fx_sincere
  130 => fx_cowardly
  40  => fx_bibliophile
  100 => fx_intoxicating

In [4]:
#const MXS = 100
const MXV = 9
const MNV = 1

# function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
#     #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
#     return h*(f+l) + f*(p+r+s)
# end

# function cost(a::Spud)::Int64
#     if a.a1 == ability_none
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s)
#     else
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s) + ability_cost[a.a1](a)
#     end
# end

1

In [5]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [6]:


function apply_ability(a::Spud, b::Spud)::Array{Spud}
    if a.a1 == ability_none
        return [a, b]
    else
        return ability_fx[a.a1](a, b)
    end
end


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_ability_first = compare_int_list([a.a1, a.s], [b.a1, b.s], utb)
    if a_ability_first ==1
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
    end
    if a_ability_first == -1
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
    end
    #println(a)
    #println(b)
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

eval_battle (generic function with 1 method)

In [7]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)


const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat(ability::Int64 = ability_none)::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    if ability == ability_none
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
        vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
        vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
        vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
        vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
        vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    else
        adj = ability_name[ability]
    end
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6], ability_none)
end

random_name_and_stat (generic function with 2 methods)

In [8]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat(a.a1)
    for ii in 1:n_tries
        b = random_name_and_stat(a.a1)
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s, a.a1)
end

rand_rename (generic function with 2 methods)

In [9]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

## Get spuds with cost 15 and filter nondominated

In [10]:
function get_library(cost, n_init = 10000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for h in hrange
        for f in frange
            for l in lrange
                for p in prange
                    for r in rrange
                        for s in srange
                            if (h+f+l+p+r+s == cost)
                                ff = Spud("", h,f,l,p,r,s,999)
                                ff_i += 1
                                ffs[ff_i] = ff
                            end
                        end
                    end                        
                end                        
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

get_library (generic function with 2 methods)

In [11]:
lib15 = get_library(15)
lib15;

In [12]:
function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

get_payoffs (generic function with 1 method)

In [13]:
function filter_nondominated2(as::Array{Spud})::Array{Spud}
    mat = get_payoffs(as);
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:length(as)
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

filter_nondominated2 (generic function with 1 method)

In [14]:
@time lib=filter_nondominated2(lib15)
# for ff in lib
#     print(" ")
#     print(100000 * ff.h + 10000 * ff.f + 1000 * ff.l + 100 * ff.p + 10 * ff.r + ff.s)
# end

 35.699681 seconds (72.09 M allocations: 125.258 GiB, 35.32% gc time, 0.96% compilation time)


1396-element Vector{Spud}:
 Spud("", 1, 1, 1, 1, 2, 9, 999)
 Spud("", 1, 1, 1, 1, 3, 8, 999)
 Spud("", 1, 1, 1, 1, 4, 7, 999)
 Spud("", 1, 1, 1, 1, 5, 6, 999)
 Spud("", 1, 1, 1, 1, 6, 5, 999)
 Spud("", 1, 1, 1, 1, 7, 4, 999)
 Spud("", 1, 1, 1, 1, 8, 3, 999)
 Spud("", 1, 1, 1, 1, 9, 2, 999)
 Spud("", 1, 1, 1, 2, 1, 9, 999)
 Spud("", 1, 1, 1, 2, 2, 8, 999)
 Spud("", 1, 1, 1, 2, 3, 7, 999)
 Spud("", 1, 1, 1, 2, 4, 6, 999)
 Spud("", 1, 1, 1, 2, 5, 5, 999)
 ⋮
 Spud("", 7, 1, 3, 1, 1, 2, 999)
 Spud("", 7, 1, 3, 1, 2, 1, 999)
 Spud("", 7, 1, 3, 2, 1, 1, 999)
 Spud("", 7, 1, 4, 1, 1, 1, 999)
 Spud("", 7, 2, 2, 1, 1, 2, 999)
 Spud("", 7, 2, 2, 1, 2, 1, 999)
 Spud("", 7, 2, 2, 2, 1, 1, 999)
 Spud("", 8, 1, 2, 1, 1, 2, 999)
 Spud("", 8, 1, 2, 1, 2, 1, 999)
 Spud("", 8, 1, 2, 2, 1, 1, 999)
 Spud("", 8, 1, 3, 1, 1, 1, 999)
 Spud("", 9, 1, 2, 1, 1, 1, 999)

In [15]:
payoffs = get_payoffs(lib)

1396×1396 Matrix{Int64}:
 0  -1  -1  -1  -1  -1  -1  -1  -1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   0  -1  -1  -1  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   0  -1  -1  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   0  -1  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   0  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   1   0  -1  -1  -1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   1   1   0  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   1   1   1   0  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   1   1   1   1   0  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   1   1   1   1   1   0     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   1   1   1   1   1   1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   1   1   1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 1   1   1   1   1   1   1   1   1   1 

In [16]:
s2ind = Dict(lib[i] => i for i in 1:length(lib));

In [17]:
s2ind[lib[5]]

5

In [18]:
using JuMP
using HiGHS

In [19]:
vector_model = Model(HiGHS.Optimizer)
n_nash = length(lib)
sumv = zeros(Int64, (1, 2 * n_nash + 1))
sumv[1:n_nash] .= 1
A = hcat(payoffs, Matrix(1I,n_nash, n_nash), -ones(Int64, n_nash, 1))
A = vcat(A, sumv);
b = zeros(Int64, n_nash + 1);
b[1:n_nash] .= -1
b[n_nash+1] = 1
transpose(b);
c = zeros(Int64, 2 * n_nash + 1);
c[2 * n_nash + 1] = 1;
@variable(vector_model, x[1:(2*n_nash+1)] >= 0);
@constraint(vector_model, A * x .== b);
@objective(vector_model, Min, c' * x)
@time optimize!(vector_model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
1397 rows, 2793 cols, 1951608 nonzeros
1397 rows, 2793 cols, 1951608 nonzeros
Presolve : Reductions: rows 1397(-0); columns 2793(-0); elements 1951608(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1397(1397) 0s
       2920     1.0000000000e+00 Pr: 0(0) 2s
       2920     1.0000000000e+00 Pr: 0(0) 2s
Model   status      : Optimal
Simplex   iterations: 2920
Objective value     :  1.0000000000e+00
HiGHS run time      :          2.86
  3.105955 seconds (237.19 k allocations: 98.574 MiB, 0.83% gc time, 5.06% compilation time: 100% of which was recompilation)


In [20]:
w_lp = [value(x[i]) for i in 1:n_nash]
sum(w_lp)

0.9999999999999993

In [21]:
w_lp = w_lp./sum(w_lp);
maximum([sum(payoffs[i, :] .* w_lp) for i in 1:n_nash])

2.886579864025407e-15

In [22]:
new_env = lib[w_lp .> 0]
w_new = w_lp[w_lp .> 0];

In [23]:
function spud2int(a::Spud)::Int64
    return a.h * 100000 + a.f * 10000 + a.l * 1000 + a.p * 100 + a.r * 10 + a.s
end

for i in sortperm(-w_new)
    print(spud2int(new_env[i]))
    print(" ")
    println(string(w_new[i])[1:6])
end

615111 0.0691
417111 0.0688
516111 0.0680
714111 0.0626
813111 0.0583
318111 0.0572
219111 0.0469
131154 0.0244
121614 0.0238
171123 0.0179
161232 0.0170
171132 0.0167
115413 0.0165
151431 0.0160
151341 0.0155
141252 0.0152
151143 0.0151
151521 0.0125
171411 0.0124
161313 0.0124
161331 0.0122
141441 0.0120
131613 0.0118
161421 0.0116
141513 0.0113
181221 0.0109
151332 0.0107
161412 0.0106
151242 0.0105
141612 0.0103
614112 0.0095
131622 0.0091
111165 0.0090
121425 0.0090
121245 0.0088
171321 0.0087
131541 0.0086
171213 0.0086
141153 0.0085
115134 0.0080
161322 0.0080
151512 0.0074
171312 0.0071
141423 0.0070
216123 0.0066
151224 0.0065
151314 0.0064
141414 0.0062
121155 0.0058
115224 0.0055
116133 0.0053
161133 0.0051
141324 0.0050
713112 0.0049
113235 0.0048
131244 0.0047
141531 0.0043
181311 0.0040
214134 0.0038
121335 0.0038
116223 0.0038
131424 0.0037
141522 0.0037
111255 0.0031
131532 0.0029
317112 0.0023
515112 0.0022
161142 0.0022
181122 0.0021
161223 0.0018
141351 0.0017
171231

In [24]:
function spud_to_vec(a::Spud)::Array{Int64}
    return [a.h, a.f, a.l, a.p, a.r, a.s]
end

function stat_dominates(a::Spud, b::Spud)::Bool
    if minimum(spud_to_vec(a) .- spud_to_vec(b))==0
        return true
    else
        return false
    end
end

function stat_dominates_any(a::Spud, bs::Array{Spud})::Bool
    for b in bs
        if stat_dominates(a, b)
            return true
        end
    end
    return false
end

stat_dominates_any (generic function with 1 method)

In [25]:
libx = get_library(15)
evs = [eval_battle_list2(ff,new_env,w_new) for ff in libx];
libp = libx[evs .< 0 .&& evs .> -0.05]
lib = unique(vcat(new_env, libp))
for ff in lib
    print(spud2int(ff))
    print(", ")
end

111165, 111255, 113145, 113235, 113325, 114135, 114225, 114234, 115134, 115224, 115413, 116133, 116223, 116412, 121155, 121245, 121335, 121425, 121614, 123414, 131154, 131244, 131424, 131532, 131541, 131613, 131622, 141153, 141252, 141324, 141351, 141414, 141423, 141441, 141513, 141522, 141531, 141612, 151143, 151224, 151242, 151314, 151332, 151341, 151431, 151512, 151521, 161133, 161142, 161223, 161232, 161313, 161322, 161331, 161412, 161421, 171123, 171132, 171213, 171231, 171312, 171321, 171411, 181113, 181122, 181221, 181311, 214134, 216123, 217131, 219111, 317112, 318111, 417111, 515112, 516111, 614112, 615111, 713112, 714111, 813111, 111345, 111435, 111525, 111714, 112155, 112245, 112335, 112425, 112614, 113334, 113415, 113424, 113514, 114144, 114324, 114414, 114513, 115233, 115314, 115323, 116124, 116214, 116232, 116313, 116322, 117114, 117123, 117213, 117312, 118113, 118212, 121164, 121254, 121344, 121434, 121515, 121524, 121533, 121623, 121632, 121641, 122145, 122235, 122325, 

In [26]:
libx = get_library(16)
evs = [eval_battle_list2(ff,new_env,w_new) for ff in libx];
libp = libx[evs .< 0 .&& evs .> -0.05]
libp = [ff for ff in libp if !stat_dominates_any(ff, lib)]
lib = vcat(lib, libp)
for ff in libp
    print(spud2int(ff))
    print(", ")
end

111364, 111454, 111616, 111634, 111661, 112516, 113164, 113533, 113623, 114163, 114343, 114532, 114622, 115126, 115252, 115612, 115621, 116152, 117241, 117511, 118231, 119122, 119131, 119221, 121453, 121462, 121552, 123253, 123343, 123541, 123622, 123631, 124153, 124252, 124342, 124612, 125242, 125341, 125521, 126142, 126331, 127231, 128122, 128221, 132442, 133252, 133342, 133441, 133612, 133621, 134143, 134242, 134341, 134521, 135331, 136321, 137122, 137221, 141136, 142126, 143242, 143341, 144331, 144511, 145132, 145222, 145321, 146122, 146221, 151162, 153115, 153142, 153331, 153511, 154132, 154222, 154321, 154411, 155122, 155221, 163132, 163231, 164122, 164221, 211444, 211534, 212344, 212434, 212515, 212524, 213253, 213343, 213433, 214252, 214315, 214342, 214432, 215251, 215341, 215431, 215512, 221362, 221452, 221461, 222253, 222343, 222523, 222532, 222613, 223252, 223315, 223342, 223432, 223513, 224251, 224341, 224422, 225214, 225241, 226231, 226321, 226411, 232342, 232513, 233242, 

In [27]:
libx = get_library(17)
evs = [eval_battle_list2(ff,new_env,w_new) for ff in libx];
libp = libx[evs .< 0 .&& evs .> -0.05]
libp = [ff for ff in libp if !stat_dominates_any(ff, lib)]
lib = vcat(lib, libp)
for ff in libp
    print(spud2int(ff))
    print(", ")
end

111563, 111653, 112544, 113363, 113453, 113462, 113642, 113651, 117161, 118151, 125261, 126116, 126611, 134162, 135116, 135611, 143162, 144116, 184112, 211463, 213362, 213452, 213461, 213632, 214541, 214631, 216116, 222551, 224126, 224621, 235115, 235511, 252611, 312353, 313262, 313352, 313442, 313532, 314441, 314612, 322262, 322352, 322361, 323261, 323351, 323441, 323531, 324161, 324521, 325115, 325511, 332261, 332441, 332531, 333431, 333512, 333521, 334421, 342521, 343421, 344411, 352511, 353411, 412235, 413162, 413225, 413351, 413441, 413522, 414215, 414431, 415511, 422252, 422351, 422441, 422522, 423215, 423251, 423431, 423512, 424115, 424421, 432215, 432251, 432431, 432512, 433151, 433241, 434321, 434411, 442421, 443231, 443321, 443411, 444221, 452411, 453221, 453311, 512225, 512315, 513215, 513251, 513512, 514421, 522251, 522431, 522512, 523151, 523421, 524411, 532115, 532151, 532421, 533411, 542411, 543221, 612242, 612332, 612341, 613421, 622412, 622421, 623411, 632411, 712241, 

In [28]:
libx = get_library(18)
evs = [eval_battle_list2(ff,new_env,w_new) for ff in libx];
libp = libx[evs .< 0 .&& evs .> -0.05]
libp = [ff for ff in libp if !stat_dominates_any(ff, lib)]
lib = vcat(lib, libp)
for ff in libp
    print(spud2int(ff))
    print(", ")
end

313551, 313641, 343611, 344151, 414621, 422631, 432621, 442611, 512352, 512451, 513531, 562131, 563121, 

In [29]:
libx = get_library(19)
evs = [eval_battle_list2(ff,new_env,w_new) for ff in libx];
libp = libx[evs .< 0 .&& evs .> -0.05]
libp = [ff for ff in libp if !stat_dominates_any(ff, lib)]
lib = vcat(lib, libp)
for ff in libp
    print(spud2int(ff))
    print(", ")
end

312562, 412462, 412552, 412561, 463141, 464131, 465121, 671122, 

In [30]:
libx = get_library(20, 20000)
evs = [eval_battle_list2(ff,new_env,w_new) for ff in libx];
libp = libx[evs .< 0 .&& evs .> -0.05]
libp = [ff for ff in libp if !stat_dominates_any(ff, lib)]
lib = vcat(lib, libp)
for ff in libp
    print(spud2int(ff))
    print(", ")
end

158141, 276131, 277121, 375131, 376121, 651161, 761141, 

In [31]:
libx = get_library(21, 20000)
evs = [eval_battle_list2(ff,new_env,w_new) for ff in libx];
libp = libx[evs .< 0 .&& evs .> -0.05]
libp = [ff for ff in libp if !stat_dominates_any(ff, lib)]
lib = vcat(lib, libp)
for ff in libp
    print(spud2int(ff))
    print(", ")
end

179121, 871131, 

In [32]:
libx = get_library(22, 20000)
evs = [eval_battle_list2(ff,new_env,w_new) for ff in libx];
libp = libx[evs .< 0 .&& evs .> -0.05]
libp = [ff for ff in libp if !stat_dominates_any(ff, lib)]
lib = vcat(lib, libp)
for ff in libp
    print(spud2int(ff))
    print(", ")
end

981121, 

In [35]:
libx = get_library(24, 30000)
evs = [eval_battle_list2(ff,new_env,w_new) for ff in libx];
libp = libx[evs .< 0 .&& evs .> -0.05]
libp = [ff for ff in libp if !stat_dominates_any(ff, lib)]
lib = vcat(lib, libp)
for ff in libp
    print(spud2int(ff))
    print(", ")
end

In [38]:
length(unique(lib))

712

## Combo analysis

In [27]:
payoffs = get_payoffs(lib)
nlib = length(lib)
libi = map(spud2int, lib);

In [ ]:
combos = Array{Int64}(undef, (10000,3))
ord = sortperm([rand() for i in 1:nlib])
#ord = [i for i in 1:nlib]
nc = 0
# 3-combos
for i in ord[1:(nlib-2)]
    print(i)
    print(" ")
    pi = payoffs[i,:]
    for j in ord[(i+1):(nlib-1)]
        pj = payoffs[j, :]
        pij = [max(pi[a], pj[a]) for a in 1:nlib]
        if sum(pij .== 1) > 0
            #print(".")
            for k in ord[(j+1):nlib]
                pk = payoffs[k, :]
                pijk = [max(pi[a], pj[a], pk[a]) for a in 1:nlib]
                if minimum(pijk)==1
                    nc += 1
                    combos[nc, :] = libi[[i,j,k]]
                    println("")
                    println("----3-COMBO!-----")
                    println(libi[[i,j,k]])
                end
            end
        end
    end
end
combos = combos[1:nc,:]

In [ ]:
size(combos)